In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
# Imports

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
from torchvision import datasets, transforms, models
from torchvision.transforms import ToTensor
from torchvision.datasets import ImageFolder

# Datasets
from datasets import load_dataset
import kagglehub

# Extra
import os
from copy import deepcopy
from PIL import Image
import matplotlib.pyplot as plt

## Office-31

In [ ]:
def Office31(domain, transform):
    path = kagglehub.dataset_download("xixuhu/office31")
    path = os.path.join(os.path.join(path, "Office-31"), domain)
    return datasets.ImageFolder(root=path, transform=transform)

def get_office_data_loaders(batch_size):

    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    amazon_data = Office31("amazon", transform)
    dslr_data = Office31("dslr", transform)
    webcam_data = Office31("webcam", transform)

    loader_amazon = DataLoader(amazon_data, batch_size=batch_size, shuffle=True)
    loader_dslr = DataLoader(dslr_data, batch_size=batch_size, shuffle=True)
    loader_webcam = DataLoader(webcam_data, batch_size=batch_size, shuffle=True)

    return loader_amazon, loader_dslr, loader_webcam

In [ ]:
loader_amazon, loader_dslr, loader_webcam = get_office_data_loaders(batch_size=32)

## Pacs

## Model Cards

In [ ]:
class ResNetFeatureExtractor(nn.Module):
    def __init__(self, pretrained=True):
        super().__init__()
        base_model = torchvision.models.resnet50(pretrained=pretrained)
        self.conv1 = base_model.conv1
        self.bn1 = base_model.bn1
        self.relu = base_model.relu
        self.maxpool = base_model.maxpool
        self.layer1 = base_model.layer1
        self.layer2 = base_model.layer2
        self.layer3 = base_model.layer3
        self.layer4 = base_model.layer4

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        f_block1 = self.layer1(x)
        f_block2 = self.layer2(f_block1)
        f_block3 = self.layer3(f_block2)
        f_block4 = self.layer4(f_block3)

        return f_block2, f_block3, f_block4

In [ ]:
class SparseAutoencoder(nn.Module):
    def __init__(self, input_dim=2048, hidden_dim=256):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(inplace=True),
        )
        self.decoder = nn.Sequential(
            nn.Linear(hidden_dim, input_dim)
        )

    def forward(self, x):
        z = self.encoder(x)
        x_recon = self.decoder(z)
        return x_recon, z

In [ ]:
class ClassifierHead(nn.Module):
    def __init__(self, input_dim=2048, num_classes=7):
        super().__init__()
        self.fc = nn.Linear(input_dim, num_classes)

    def forward(self, x):
        return self.fc(x)

In [ ]:
class BaselineModel(nn.Module):
    def __init__(self, feature_extractor, num_classes=7):

        super().__init__()
        self.feature_extractor = feature_extractor
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

        self.classifier = nn.Linear(2048, num_classes)

    def forward(self, x):
        _, _, f4 = self.feature_extractor(x)
        f4_pool = self.avgpool(f4).view(f4.size(0), -1)

        logits = self.classifier(f4_pool)
        return logits, f4_pool


In [ ]:
class UnifiedModelMultiBlockSAE(nn.Module):
    def __init__(self,
                 sae_dim_block2=128,
                 sae_dim_block3=256,
                 sae_dim_block4=256,
                 num_classes=7):

        super().__init__()
        self.feature_extractor = ResNetFeatureExtractor(pretrained=True)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

        self.sae2 = SparseAutoencoder(input_dim=512, hidden_dim=sae_dim_block2)
        self.sae3 = SparseAutoencoder(input_dim=1024, hidden_dim=sae_dim_block3)
        self.sae4 = SparseAutoencoder(input_dim=2048, hidden_dim=sae_dim_block4)

        total_hidden = sae_dim_block2 + sae_dim_block3 + sae_dim_block4
        self.classifier = nn.Linear(total_hidden, num_classes)

    def forward(self, x):

        f2, f3, f4 = self.feature_extractor(x)

        f2_pool = self.avgpool(f2).view(f2.size(0), -1)
        f3_pool = self.avgpool(f3).view(f3.size(0), -1)
        f4_pool = self.avgpool(f4).view(f4.size(0), -1)

        x_recon2, z2 = self.sae2(f2_pool)
        x_recon3, z3 = self.sae3(f3_pool)
        x_recon4, z4 = self.sae4(f4_pool)

        z_concat = torch.cat([z2, z3, z4], dim=1)

        class_logits = self.classifier(z_concat)

        return (class_logits,
                (x_recon2, z2),
                (x_recon3, z3),
                (x_recon4, z4),
                (f2_pool, f3_pool, f4_pool))

## Losses

In [ ]:
def irm_penalty(logits, labels):

    scale = torch.tensor(1.0, requires_grad=True, device=logits.device)
    loss_erm = F.cross_entropy(scale * logits, labels)

    grad = torch.autograd.grad(loss_erm, [scale], create_graph=True)[0]

    penalty = torch.sum(grad**2)

    return loss_erm, penalty

In [ ]:
def compute_sae_loss(recons, pools, zs):

    x2_recon, x3_recon, x4_recon = recons
    f2p, f3p, f4p = pools
    z2, z3, z4 = zs

    rec_loss2 = F.mse_loss(x2_recon, f2p)
    rec_loss3 = F.mse_loss(x3_recon, f3p)
    rec_loss4 = F.mse_loss(x4_recon, f4p)
    recon_loss_total = (rec_loss2 + rec_loss3 + rec_loss4)

    l1_z2 = torch.mean(torch.abs(z2))
    l1_z3 = torch.mean(torch.abs(z3))
    l1_z4 = torch.mean(torch.abs(z4))
    l1_sparsity_total = (l1_z2 + l1_z3 + l1_z4)

    return recon_loss_total, l1_sparsity_total


## Trian and Evaluate Functions

### Train

In [ ]:
def train_warmup(model, loader_source_domain1, loader_source_domain2, loader_test,
                 num_warmup_epochs=5,
                 lr=1e-4,
                 device='cuda'):

    model.to(device)
    model.train()


    optimizer = optim.Adam([
        {'params': model.feature_extractor.parameters(), 'lr': lr},
        {'params': model.classifier.parameters(), 'lr': lr},
    ], lr=lr)

    for epoch in range(num_warmup_epochs):

        domain1_iter = iter(loader_source_domain1)
        domain2_iter = iter(loader_source_domain2)
        steps_per_epoch = max(len(domain1_iter), len(domain2_iter))

        total_loss = 0.0
        total_samples = 0

        for step in range(steps_per_epoch):
            try:
                x_s1, y_s1 = next(domain1_iter)
            except StopIteration:
                domain1_iter = iter(loader_source_domain1)
                x_s1, y_s1 = next(domain1_iter)

            try:
                x_s2, y_s2 = next(domain2_iter)
            except StopIteration:
                domain2_iter = iter(loader_source_domain2)
                x_s2, y_s2 = next(domain2_iter)


            x_source = torch.cat([x_s1, x_s2], dim=0)
            y_source = torch.cat([y_s1, y_s2], dim=0)

            indices = torch.randperm(x_source.size(0))
            x_source = x_source[indices]
            y_source = y_source[indices]

            x_source, y_source = x_source.to(device), y_source.to(device)

            class_logits, _, _, _, _ = model(x_source, grl_enabled=False)
            loss_cls = F.cross_entropy(class_logits, y_source)

            optimizer.zero_grad()
            loss_cls.backward()
            optimizer.step()

            bs = x_source.size(0)
            total_loss += loss_cls.item() * bs
            total_samples += bs

        epoch_loss = total_loss / total_samples
        print(f"[Warmup] Epoch [{epoch+1}/{num_warmup_epochs}] Loss: {epoch_loss:.4f}")
        test_acc = evaluate(model, loader_test, device=device)
        print(f"** End of Epoch {epoch+1}/{num_warmup_epochs} | Test Accuracy: {test_acc:.2f}% **")


In [ ]:
def train_main(model,
               loader_domain1,
               loader_domain2,
               test_loader,
               num_epochs=20,
               lr=1e-4,
               lambda_irm=1.0,
               lambda_sae_rec=1.0,
               lambda_sae_sparse=1e-4,
               device='cuda'):

    model.to(device)
    model.train()

    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(num_epochs):

        d1_iter = iter(loader_domain1)
        d2_iter   = iter(loader_domain2)

        steps_per_epoch = max(len(d1_iter), len(d2_iter))

        for step in range(steps_per_epoch):

            try:
                x_s1, y_s1 = next(d1_iter)
            except StopIteration:
                d1_iter = iter(loader_domain1)
                x_s1, y_s1 = next(d1_iter)

            try:
                x_s2, y_s2 = next(d2_iter)
            except StopIteration:
                d2_iter = iter(loader_domain2)
                x_s2, y_s2 = next(d2_iter)

            x_s1, y_s1 = x_s1.to(device), y_s1.to(device)
            x_s2, y_s2 = x_s2.to(device), y_s2.to(device)


            logits_s1, _, _, _, _ = model(x_s1, grl_enabled=False)
            logits_s2, _, _, _, _ = model(x_s2, grl_enabled=False)

            loss_erm_s1, penalty_s1 = irm_penalty(logits_s1, y_s1)
            loss_erm_s2, penalty_s2 = irm_penalty(logits_s2,   y_s2)

            irm_loss = 0.5 * (loss_erm_s1 + loss_erm_s2)
            irm_pen  = 0.5 * (penalty_s1 + penalty_s2)
            loss_irm = irm_loss + lambda_irm * irm_pen


            with torch.no_grad():
                _, _, _, _, f4p_s1 = model(x_s1, grl_enabled=False)
                _, _, _, _, f4p_s2 = model(x_s2, grl_enabled=False)


            x_recon_s1, z_s1 = model.sae(f4p_s1)
            x_recon_s2, z_s2 = model.sae(f4p_s2)

            recon_loss_s1 = F.mse_loss(x_recon_s1, f4p_s1)
            recon_loss_s2 = F.mse_loss(x_recon_s2, f4p_s2)
            recon_loss   = 0.5 * (recon_loss_s1 + recon_loss_s2)

            l1_sparsity_s1 = torch.mean(torch.abs(z_s1))
            l1_sparsity_s2 = torch.mean(torch.abs(z_s2))
            l1_sparsity   = 0.5 * (l1_sparsity_s1 + l1_sparsity_s2)

            sae_loss = lambda_sae_rec * recon_loss + lambda_sae_sparse * l1_sparsity

            loss = loss_irm + sae_loss

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (step + 1) % 10 == 0:
                print(f"[Main] Epoch [{epoch+1}/{num_epochs}], Step [{step+1}/{steps_per_epoch}], "
                      f"IRM Loss: {loss_irm.item():.4f}, SAE Loss: {sae_loss.item():.4f}")

        test_acc = evaluate(model, test_loader, device=device)
        print(f"** End of Epoch {epoch+1}/{num_epochs} | Test Accuracy: {test_acc:.2f}% **")

    return model


In [ ]:
def train_baseline_model_office(model,
                         loader_source,
                         target_loader,
                         num_epochs=20,
                         lr=1e-4,
                         lambda_irm=1.0,
                         device='cuda'):

    model = model.to(device)
    model.train()

    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(num_epochs):

        source_iter = iter(loader_source)
        steps_per_epoch = len(source_iter)
        for step in range(steps_per_epoch):

            try:
                x_s, y_s = next(source_iter)
            except StopIteration:
                source_iter = iter(loader_source)
                x_s, y_s = next(source_iter)

            x_s, y_s = x_s.to(device), y_s.to(device)

            logits_s, _ = model(x_s)
            loss_erm_s, penalty_s = irm_penalty(logits_s, y_s)

            irm_loss = 0.5 * (loss_erm_s)
            irm_pen  = 0.5 * (penalty_s)
            loss_irm = irm_loss + lambda_irm * irm_pen

            optimizer.zero_grad()
            loss_irm.backward()
            optimizer.step()

            if (step+1) % 10 == 0:
                print(f"[Baseline IRM] Epoch [{epoch+1}/{num_epochs}], "
                      f"Step [{step+1}/{steps_per_epoch}], IRM Loss: {loss_irm.item():.4f}")

        test_acc = evaluate_baseline(model, target_loader, device=device)
        print(f"** End of Epoch {epoch+1}/{num_epochs} | Test Accuracy: {test_acc:.2f}% **")

    return model

In [ ]:
def train_main_irm_multi_sae_office(model,
                              loader_source,
                              test_loader,
                              num_epochs=20,
                              lr=1e-4,
                              lambda_irm=1.0,
                              lambda_sae_rec=1.0,
                              lambda_sae_sparse=1e-4,
                              device='cuda',
                              verbose=False):

    model.to(device)
    model.train()

    def sae_forward_splits(f2p, f3p, f4p):
        x_recon2, z2 = model.sae2(f2p)
        x_recon3, z3 = model.sae3(f3p)
        x_recon4, z4 = model.sae4(f4p)
        return x_recon2, z2, x_recon3, z3, x_recon4, z4

    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(num_epochs):
        source_iter = iter(loader_source)
        steps_per_epoch = len(source_iter)

        for step in range(steps_per_epoch):

            try:
                x_s, y_s = next(source_iter)
            except StopIteration:
                source_iter = iter(loader_source)
                x_s, y_s = next(source_iter)

            x_s, y_s = x_s.to(device), y_s.to(device)

            class_logits_s, _, _, _, _ = model(x_s)

            loss_erm_s, penalty_s = irm_penalty(class_logits_s, y_s)

            irm_loss = 0.5 * (loss_erm_s)
            irm_pen  = 0.5 * (penalty_s)

            loss_irm = irm_loss + lambda_irm * irm_pen


            with torch.no_grad():
                _, (x_recon2_s, z2_s), (x_recon3_s, z3_s), (x_recon4_s, z4_s), (f2p_s, f3p_s, f4p_s) = model(x_s)

            x_recon2_s, z2_s, x_recon3_s, z3_s, x_recon4_s, z4_s = sae_forward_splits(f2p_s, f3p_s, f4p_s)

            rec_loss2_s = F.mse_loss(x_recon2_s, f2p_s)
            rec_loss2   = 1 * (rec_loss2_s)

            rec_loss3_s = F.mse_loss(x_recon3_s, f3p_s)
            rec_loss3   = 1 * (rec_loss3_s)

            rec_loss4_s = F.mse_loss(x_recon4_s, f4p_s)
            rec_loss4   = 1 * (rec_loss4_s)

            rec_loss_total = (rec_loss2 + rec_loss3 + rec_loss4)

            l1_2_s = torch.mean(torch.abs(z2_s))
            l1_2   = 1 * (l1_2_s)

            l1_3_s = torch.mean(torch.abs(z3_s))
            l1_3   = 1 * (l1_3_s)

            l1_4_s = torch.mean(torch.abs(z4_s))
            l1_4   = 1 * (l1_4_s)

            l1_sparsity = l1_2 + l1_3 + l1_4

            sae_loss = lambda_sae_rec * rec_loss_total + lambda_sae_sparse * l1_sparsity

            loss = loss_irm + sae_loss

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (step+1) % 40 == 0 and verbose:
                print(f"Epoch [{epoch+1}/{num_epochs}], Step [{step+1}/{steps_per_epoch}], "
                      f"IRM Loss: {loss_irm.item():.4f}, SAE Loss: {sae_loss.item():.4f}")

        test_acc = evaluate(model, test_loader, device=device)
        print(f"** End of Epoch {epoch+1}/{num_epochs} | Test Accuracy: {test_acc:.2f}% **")

    return model

In [ ]:
def train_model_irm_sae_with_warmup_office(
    batch_size=32,
    num_warmup_epochs=5,
    num_main_epochs=20,
    lr=1e-4,
    lambda_irm=1.0,
    lambda_sae_rec=1.0,
    lambda_sae_sparse=1e-4,
    device='cuda',
    loader=['A', 'W'],
    model=None,
    verbose=False
):

    loader_amazon, loader_webcam, loader_dslr = get_office_data_loaders(batch_size)

    source, target = loader if loader is not None else ['A', 'W']

    if source == 'A':
        loader_source = loader_amazon
    elif source == 'W':
        loader_source = loader_webcam
    elif source == 'D':
        loader_source = loader_dslr

    if target == 'A':
        loader_target = loader_amazon
    elif target == 'W':
        loader_target = loader_webcam
    elif target == 'D':
        loader_target = loader_dslr


    if model is None:
        model = UnifiedModelMultiBlockSAE(512, 1024, 2048, 31)

    print("===== Main Phase (IRM + SAE) =====")

    model = train_main_irm_multi_sae_office(model,
                                            loader_source=loader_source,
                                            test_loader=loader_target,
                                            num_epochs=num_main_epochs,
                                            lr=lr,
                                            lambda_irm=lambda_irm,
                                            lambda_sae_rec=lambda_sae_rec,
                                            lambda_sae_sparse=lambda_sae_sparse,
                                            device=device,
                                            verbose=verbose)

    return model

### Evaluate

In [ ]:
def evaluate_baseline(model, loader, device='cuda'):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            logits, _ = model(x)
            preds = torch.argmax(logits, dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)
    acc = 100.0 * correct / total
    model.train()
    return acc


In [ ]:
def evaluate(model, loader, device='cuda'):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            logits, _, _, _, _ = model(x)
            preds = torch.argmax(logits, dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)
    acc = 100.0 * correct / total
    return acc

# MAIN

In [ ]:
feature_extractor_baseline = ResNetFeatureExtractor(pretrained=True)
baseline_model = BaselineModel(feature_extractor=feature_extractor_baseline, num_classes=31)

In [ ]:
loader_amazon, loader_webcam, loader_dslr = get_office_data_loaders(32)

### Baseline models

#### A -> W

In [ ]:
baseline_model = train_baseline_model_office(baseline_model,
                         loader_amazon,
                         loader_webcam,
                         num_epochs=20,
                         lr=3e-5,
                         lambda_irm=4.0,
                         device='cuda')

#### A -> D

In [ ]:
baseline_model = train_baseline_model_office(baseline_model,
                         loader_amazon,
                         loader_dslr,
                         num_epochs=20,
                         lr=3e-5,
                         lambda_irm=4.0,
                         device='cuda')

#### W -> A

In [ ]:
baseline_model = train_baseline_model_office(baseline_model,
                         loader_webcam,
                         loader_amazon,
                         num_epochs=20,
                         lr=3e-5,
                         lambda_irm=4.0,
                         device='cuda')

#### W -> D

In [ ]:
baseline_model = train_baseline_model_office(baseline_model,
                         loader_webcam,
                         loader_dslr,
                         num_epochs=20,
                         lr=3e-5,
                         lambda_irm=4.0,
                         device='cuda')

#### D -> W

In [ ]:
baseline_model = train_baseline_model_office(baseline_model,
                         loader_dslr,
                         loader_webcam,
                         num_epochs=20,
                         lr=3e-5,
                         lambda_irm=4.0,
                         device='cuda')

#### D -> A

In [ ]:
baseline_model = train_baseline_model_office(baseline_model,
                         loader_dslr,
                         loader_amazon,
                         num_epochs=20,
                         lr=3e-5,
                         lambda_irm=4.0,
                         device='cuda')

### Ours

In [ ]:
combinations = [["A", "W"], ["A", "D"], ["W", "A"], ["W", "D"], ["D", "W"], ["D", "A"]]

In [ ]:
for combination in combinations:
    print(f"------------Combination: {combination}--------------")
    trained_model_sae_office = train_model_irm_sae_with_warmup_office(
            batch_size=32,
            num_warmup_epochs=4,
            num_main_epochs=30,
            lr=2e-5,
            lambda_irm=4.0,
            lambda_sae_rec=2.0,
            lambda_sae_sparse=2e-4,
            device='cuda',
            loader=combination,
            verbose=False
        )
    print("--------------------------------------------")


------------Combination: ['A', 'W']--------------
===== Main Phase (IRM + SAE) =====
** End of Epoch 1/30 | Test Accuracy: 8.43% **
** End of Epoch 2/30 | Test Accuracy: 12.85% **
** End of Epoch 3/30 | Test Accuracy: 27.51% **
** End of Epoch 4/30 | Test Accuracy: 40.96% **
** End of Epoch 5/30 | Test Accuracy: 50.40% **
** End of Epoch 6/30 | Test Accuracy: 56.83% **
** End of Epoch 7/30 | Test Accuracy: 60.04% **
** End of Epoch 8/30 | Test Accuracy: 66.06% **
** End of Epoch 9/30 | Test Accuracy: 70.08% **
** End of Epoch 10/30 | Test Accuracy: 69.08% **
** End of Epoch 11/30 | Test Accuracy: 74.70% **
** End of Epoch 12/30 | Test Accuracy: 74.90% **
** End of Epoch 13/30 | Test Accuracy: 73.29% **
** End of Epoch 14/30 | Test Accuracy: 76.31% **
** End of Epoch 15/30 | Test Accuracy: 69.08% **
** End of Epoch 16/30 | Test Accuracy: 69.88% **
** End of Epoch 17/30 | Test Accuracy: 70.68% **
** End of Epoch 18/30 | Test Accuracy: 72.49% **
** End of Epoch 19/30 | Test Accuracy: 72.6

In [ ]:
combinations = [["A", "D"], ["W", "A"]]

In [ ]:
for combination in combinations:
    print(f"------------Combination: {combination}--------------")
    trained_model_sae_office = train_model_irm_sae_with_warmup_office(
            batch_size=32,
            num_warmup_epochs=4,
            num_main_epochs=30,
            lr=2e-5,
            lambda_irm=4.0,
            lambda_sae_rec=3.0,
            lambda_sae_sparse=3e-4,
            device='cuda',
            loader=combination,
            verbose=False
        )
    print("--------------------------------------------")


------------Combination: ['A', 'D']--------------


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


===== Main Phase (IRM + SAE) =====
** End of Epoch 1/30 | Test Accuracy: 4.65% **
** End of Epoch 2/30 | Test Accuracy: 14.84% **
** End of Epoch 3/30 | Test Accuracy: 38.74% **
** End of Epoch 4/30 | Test Accuracy: 44.40% **
** End of Epoch 5/30 | Test Accuracy: 49.94% **
** End of Epoch 6/30 | Test Accuracy: 61.76% **
** End of Epoch 7/30 | Test Accuracy: 61.01% **
** End of Epoch 8/30 | Test Accuracy: 63.02% **
** End of Epoch 9/30 | Test Accuracy: 63.52% **
** End of Epoch 10/30 | Test Accuracy: 54.47% **
** End of Epoch 11/30 | Test Accuracy: 63.90% **
** End of Epoch 12/30 | Test Accuracy: 64.40% **
** End of Epoch 13/30 | Test Accuracy: 63.65% **
** End of Epoch 14/30 | Test Accuracy: 64.03% **
** End of Epoch 15/30 | Test Accuracy: 65.79% **
** End of Epoch 16/30 | Test Accuracy: 65.03% **
** End of Epoch 17/30 | Test Accuracy: 66.42% **
** End of Epoch 18/30 | Test Accuracy: 63.40% **
** End of Epoch 19/30 | Test Accuracy: 62.26% **
** End of Epoch 20/30 | Test Accuracy: 62.01